In [19]:
import pandas as pd
import numpy as np
import re
from time import time
import math
import feather
from datetime import datetime
import os
import glob
from tqdm import tqdm_notebook as tqdm
import optuna
from optuna.integration import KerasPruningCallback
import tempfile
import tensorflow as tf
import random as rn

#Using keras
import keras
from keras import backend as K
from keras.models import Sequential  
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

#from scripts.utils import load_rnn_data
pd.options.display.max_columns = 200

# validation
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error, log_loss
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from copy import deepcopy

# others
import warnings
warnings.filterwarnings("ignore")

## Load Data

In [20]:
def load_rnn_data(path, window, predict_ts, isdim3=True, geo_col=["geoid10_tract"], y_cols=["crime"]):
    """
    y_cols: ["crime"] or ["incident_type_0", "incident_type_1", "incident_type_2"]
    geo_col: ["geoid10_tract"] or ["geoid10_block"]
    return y_all and x_all of given path
    """
    # load data
    df = feather.read_dataframe(path)
    df.sort_values(by=["datetime", "geoid10_tract"], inplace=True)
    df.set_index("datetime", inplace=True)

    # input columns
    x_cols = list(df.drop(y_cols + geo_col, axis=1).columns)

    # group by geoid
    geo_grs = df.groupby(by=geo_col)

    # arrayes to store x and y
    # (no of timesteps, window size, no of tracts,  no of features, )
    n_timesteps = int(len(df) / len(geo_grs)) - window - predict_ts + 1
    x_all = np.empty(shape=(n_timesteps, window, len(geo_grs), len(x_cols + y_cols)))

    # (output size, no of tracts, no of outputs)
    y_all = np.empty(shape=(n_timesteps, len(geo_grs), len(y_cols)))

    # to store geo_ids and y_all's datetime
    geo_ids = []

    y_datetime = df.index.unique()[window + predict_ts - 1:]

    for i, (geo_id, gr) in enumerate(tqdm(geo_grs)):
        geo_ids.append(geo_id)
        x_values = gr[y_cols + x_cols].values
        y_values = gr[y_cols].values

        for j in range(window, len(gr) - predict_ts + 1):
            # generate x_all
            x_all[j - window, :, i, :] = x_values[j - window:j, :]
            y_all[j - window, i, :] = y_values[j + predict_ts - 1, :]

    if isdim3:
        x_all = np.reshape(x_all,
                           newshape=(x_all.shape[0], x_all.shape[1], x_all.shape[2] * x_all.shape[3]))
        y_all = np.reshape(y_all,
                           newshape=(y_all.shape[0], y_all.shape[1] * y_all.shape[2]))

    return x_all, y_all, geo_ids, y_datetime

In [21]:
# set configuration
path = "./features/features_binary_tract_2H.feather"
window = 12
predict_ts = 1  # how many timesteps future does the model predict? 

In [22]:
# load data as x and y of RNN
x_all, y_all, geo_ids, y_datetime = load_rnn_data(path=path,
                                                  window=window,
                                                  predict_ts=predict_ts,
                                                  isdim3=True,
                                                  geo_col=["geoid10_tract"],
                                                  y_cols=["crime"])

In [23]:
print(x_all.shape)
print(y_all.shape)
print(len(geo_ids))  # to convert model output later
print(len(y_datetime))  # to convert model output later

(18600, 12, 1560)
(18600, 195)
195
18600


## Modeling
see https://github.com/pfnet/optuna/blob/master/examples/pruning/keras_integration.py

In [24]:
BATCHSIZE = 128
CLASSES = 10
EPOCHS = 20

In [25]:
# set seed
np.random.seed(0)
os.environ["PYTHONHASHSEED"] = "0"
rn.seed(0)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)
tf.set_random_seed(0)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [30]:
def create_model(trial):
    # We optimize the number of layers, hidden units and dropout in each layer and
    # the learning rate of RMSProp optimizer.

    # We define our MLP.
    n_layers = trial.suggest_int('n_layers', 1, 3)
    model = Sequential()
    model.add(LSTM(units=100, input_shape=(x_train.shape[1], x_train.shape[2])))
    #Hitting the first error above - how do we get the time series code to feed an example of 
    for i in range(n_layers):
        num_hidden = int(trial.suggest_loguniform('n_units_l{}'.format(i), 4, 128))
        model.add(Dense(num_hidden, activation='sigmoid'))
        dropout = trial.suggest_uniform('dropout_l{}'.format(i), 0.2, 0.5)
        model.add(Dropout(rate=dropout))
    model.add(Dense(units=y_train.shape[1], activation='softmax'))   

    # We compile our model with a sampled learning rate.
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.RMSprop(lr=lr),
                  metrics=['accuracy'])

    return model

In [27]:
## Need to define this to produce a training and test sample

def splitting(data = x_all):
    # create train and test set
    split = int(len(x_all) * 0.8)    
    x_train = x_all[:split]
    y_train = y_all[:split]
    x_test = x_all[split:]
    y_test = y_all[split:]
    return x_train, y_train, x_test, y_test

In [28]:
def objective(trial):
    # Clear clutter form previous session graphs.
    keras.backend.clear_session()
    
    # time series split
    x_train, y_train, x_test, y_test = splitting(data = x_all)

    # Generate our trial model.
    model = create_model(trial)

    # Fit the model on the training data.
    # The KerasPruningCallback checks for pruning condition every epoch.
    model.fit(x_train,
              y_train,
              batch_size=BATCHSIZE,
              callbacks=[KerasPruningCallback(trial, 'val_acc')],
              epochs=EPOCHS,
              validation_split=0.2,
              verbose=1)
    # Evaluate the model accuracy on the test set.
    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]


In [31]:
study = optuna.create_study(pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=100)
pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]
print('Study statistics: ')
print('  Number of finished trials: ', len(study.trials))
print('  Number of pruned trials: ', len(pruned_trials))
print('  Number of complete trials: ', len(complete_trials))

print('Best trial:')
trial = study.best_trial

print('  Value: ', trial.value)

print('  Params: ')
for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Instructions for updating:
Use tf.cast instead.
Train on 11904 samples, validate on 2976 samples
Epoch 1/20
11904/11904 [==============================] - ETA: 2:05 - loss: 97.2478 - acc: 0.00 - ETA: 1:17 - loss: 97.2913 - acc: 0.01 - ETA: 1:00 - loss: 97.3595 - acc: 0.01 - ETA: 53s - loss: 95.6150 - acc: 0.0215 - ETA: 47s - loss: 94.1753 - acc: 0.018 - ETA: 43s - loss: 93.3588 - acc: 0.018 - ETA: 40s - loss: 92.7684 - acc: 0.017 - ETA: 38s - loss: 91.7616 - acc: 0.019 - ETA: 36s - loss: 91.5565 - acc: 0.018 - ETA: 35s - loss: 91.5588 - acc: 0.018 - ETA: 33s - loss: 91.8941 - acc: 0.019 - ETA: 32s - loss: 92.7195 - acc: 0.019 - ETA: 31s - loss: 93.7106 - acc: 0.021 - ETA: 30s - loss: 93.2535 - acc: 0.020 - ETA: 29s - loss: 93.3420 - acc: 0.022 - ETA: 29s - loss: 93.2613 - acc: 0.022 - ETA: 28s - loss: 93.1429 - acc: 0.022 - ETA: 28s - loss: 93.0122 - acc: 0.023 - ETA: 27s - loss: 92.8556 - acc: 0.023 - ETA: 27s - loss: 93.2146 - acc: 0.023 - ETA: 26s - loss: 93.1098 - acc: 0.025 - ETA:

11904/11904 [==============================] - ETA: 34s - loss: 86.1477 - acc: 0.156 - ETA: 34s - loss: 89.8247 - acc: 0.132 - ETA: 33s - loss: 88.0889 - acc: 0.145 - ETA: 33s - loss: 88.3531 - acc: 0.140 - ETA: 33s - loss: 88.3984 - acc: 0.139 - ETA: 32s - loss: 89.0256 - acc: 0.140 - ETA: 31s - loss: 88.7680 - acc: 0.139 - ETA: 30s - loss: 88.1219 - acc: 0.138 - ETA: 29s - loss: 87.8840 - acc: 0.135 - ETA: 29s - loss: 87.8915 - acc: 0.136 - ETA: 28s - loss: 88.3634 - acc: 0.135 - ETA: 28s - loss: 88.6559 - acc: 0.135 - ETA: 27s - loss: 89.0021 - acc: 0.136 - ETA: 27s - loss: 89.2949 - acc: 0.136 - ETA: 26s - loss: 89.3539 - acc: 0.135 - ETA: 26s - loss: 89.3419 - acc: 0.134 - ETA: 26s - loss: 89.6010 - acc: 0.138 - ETA: 25s - loss: 89.3123 - acc: 0.136 - ETA: 25s - loss: 89.4863 - acc: 0.134 - ETA: 24s - loss: 89.4431 - acc: 0.133 - ETA: 24s - loss: 89.4686 - acc: 0.133 - ETA: 24s - loss: 89.4124 - acc: 0.132 - ETA: 23s - loss: 89.4569 - acc: 0.131 - ETA: 23s - loss: 89.3160 - acc: 0

11904/11904 [==============================] - ETA: 29s - loss: 88.8377 - acc: 0.140 - ETA: 27s - loss: 90.9545 - acc: 0.121 - ETA: 25s - loss: 91.5035 - acc: 0.119 - ETA: 25s - loss: 89.8343 - acc: 0.103 - ETA: 24s - loss: 90.4066 - acc: 0.106 - ETA: 24s - loss: 91.3518 - acc: 0.106 - ETA: 24s - loss: 90.9285 - acc: 0.103 - ETA: 23s - loss: 91.7149 - acc: 0.099 - ETA: 23s - loss: 91.6982 - acc: 0.099 - ETA: 23s - loss: 91.8235 - acc: 0.097 - ETA: 22s - loss: 91.4342 - acc: 0.098 - ETA: 22s - loss: 91.5352 - acc: 0.098 - ETA: 22s - loss: 91.1748 - acc: 0.096 - ETA: 21s - loss: 90.8579 - acc: 0.093 - ETA: 21s - loss: 91.1042 - acc: 0.092 - ETA: 21s - loss: 90.8886 - acc: 0.090 - ETA: 20s - loss: 90.6926 - acc: 0.089 - ETA: 20s - loss: 90.2483 - acc: 0.088 - ETA: 20s - loss: 89.9545 - acc: 0.091 - ETA: 19s - loss: 89.7648 - acc: 0.089 - ETA: 19s - loss: 89.9745 - acc: 0.087 - ETA: 19s - loss: 90.1213 - acc: 0.088 - ETA: 18s - loss: 89.7424 - acc: 0.087 - ETA: 18s - loss: 89.5462 - acc: 0

11904/11904 [==============================] - ETA: 31s - loss: 90.9034 - acc: 0.023 - ETA: 32s - loss: 89.3689 - acc: 0.023 - ETA: 30s - loss: 88.6233 - acc: 0.026 - ETA: 30s - loss: 90.4901 - acc: 0.021 - ETA: 30s - loss: 90.7030 - acc: 0.020 - ETA: 29s - loss: 90.3583 - acc: 0.019 - ETA: 29s - loss: 88.6272 - acc: 0.016 - ETA: 29s - loss: 89.2301 - acc: 0.014 - ETA: 28s - loss: 89.4088 - acc: 0.013 - ETA: 28s - loss: 88.5040 - acc: 0.014 - ETA: 28s - loss: 88.5551 - acc: 0.012 - ETA: 29s - loss: 88.3894 - acc: 0.014 - ETA: 29s - loss: 88.3089 - acc: 0.013 - ETA: 30s - loss: 88.2048 - acc: 0.013 - ETA: 30s - loss: 88.1504 - acc: 0.013 - ETA: 29s - loss: 88.0721 - acc: 0.013 - ETA: 29s - loss: 88.3123 - acc: 0.013 - ETA: 28s - loss: 88.4432 - acc: 0.013 - ETA: 27s - loss: 88.4196 - acc: 0.013 - ETA: 27s - loss: 88.4117 - acc: 0.012 - ETA: 26s - loss: 88.5075 - acc: 0.013 - ETA: 26s - loss: 88.4242 - acc: 0.013 - ETA: 26s - loss: 88.4815 - acc: 0.012 - ETA: 25s - loss: 88.4810 - acc: 0

11904/11904 [==============================] - ETA: 42s - loss: 95.2826 - acc: 0.023 - ETA: 34s - loss: 89.9982 - acc: 0.023 - ETA: 31s - loss: 90.8074 - acc: 0.015 - ETA: 29s - loss: 91.0997 - acc: 0.011 - ETA: 28s - loss: 90.3270 - acc: 0.012 - ETA: 27s - loss: 90.1414 - acc: 0.013 - ETA: 27s - loss: 90.5364 - acc: 0.014 - ETA: 27s - loss: 90.0506 - acc: 0.014 - ETA: 26s - loss: 89.5554 - acc: 0.015 - ETA: 25s - loss: 89.6259 - acc: 0.015 - ETA: 25s - loss: 89.9341 - acc: 0.014 - ETA: 24s - loss: 89.7996 - acc: 0.015 - ETA: 24s - loss: 90.0450 - acc: 0.015 - ETA: 23s - loss: 89.3512 - acc: 0.015 - ETA: 23s - loss: 89.9648 - acc: 0.017 - ETA: 23s - loss: 89.7427 - acc: 0.016 - ETA: 23s - loss: 89.5431 - acc: 0.016 - ETA: 23s - loss: 89.3344 - acc: 0.015 - ETA: 22s - loss: 89.4080 - acc: 0.014 - ETA: 22s - loss: 89.3835 - acc: 0.014 - ETA: 22s - loss: 89.1664 - acc: 0.014 - ETA: 21s - loss: 89.2765 - acc: 0.013 - ETA: 21s - loss: 88.9862 - acc: 0.013 - ETA: 21s - loss: 89.1685 - acc: 0

...

11904/11904 [==============================] - ETA: 28s - loss: 82.8329 - acc: 0.015 - ETA: 28s - loss: 88.1995 - acc: 0.007 - ETA: 29s - loss: 89.0977 - acc: 0.005 - ETA: 29s - loss: 87.3334 - acc: 0.007 - ETA: 28s - loss: 88.4415 - acc: 0.006 - ETA: 28s - loss: 87.6733 - acc: 0.005 - ETA: 27s - loss: 87.5771 - acc: 0.004 - ETA: 27s - loss: 87.4294 - acc: 0.003 - ETA: 27s - loss: 87.3791 - acc: 0.003 - ETA: 27s - loss: 87.2561 - acc: 0.003 - ETA: 26s - loss: 87.0072 - acc: 0.003 - ETA: 26s - loss: 87.1480 - acc: 0.003 - ETA: 26s - loss: 87.3039 - acc: 0.004 - ETA: 26s - loss: 87.3638 - acc: 0.004 - ETA: 25s - loss: 87.3952 - acc: 0.004 - ETA: 25s - loss: 87.8791 - acc: 0.003 - ETA: 25s - loss: 88.5062 - acc: 0.004 - ETA: 24s - loss: 88.3002 - acc: 0.003 - ETA: 24s - loss: 88.0014 - acc: 0.004 - ETA: 24s - loss: 87.9610 - acc: 0.004 - ETA: 23s - loss: 88.0073 - acc: 0.004 - ETA: 23s - loss: 87.9362 - acc: 0.005 - ETA: 23s - loss: 87.8293 - acc: 0.005 - ETA: 22s - loss: 87.6125 - acc: 0

[I 2019-05-04 16:53:45,531] Finished a trial resulted in value: 0.00215053763441. Current best value is 0.00215053763441 with parameters: {'n_units_l1': 17.088067551657936, 'n_units_l0': 126.64573450984726, 'n_layers': 2, 'lr': 0.00015707417334894226, 'dropout_l0': 0.4049191965146334, 'dropout_l1': 0.34702893287427206}.


Train on 11904 samples, validate on 2976 samples
Epoch 1/20
11904/11904 [==============================] - ETA: 2:38 - loss: 91.9796 - acc: 0.0000e+ - ETA: 1:34 - loss: 109.0968 - acc: 0.0039   - ETA: 1:12 - loss: 122.0803 - acc: 0.013 - ETA: 1:01 - loss: 127.8411 - acc: 0.009 - ETA: 54s - loss: 126.7900 - acc: 0.009 - ETA: 49s - loss: 125.9833 - acc: 0.00 - ETA: 46s - loss: 124.9159 - acc: 0.00 - ETA: 43s - loss: 123.3286 - acc: 0.01 - ETA: 41s - loss: 121.8100 - acc: 0.01 - ETA: 39s - loss: 120.8053 - acc: 0.01 - ETA: 38s - loss: 118.8243 - acc: 0.01 - ETA: 37s - loss: 116.9175 - acc: 0.01 - ETA: 36s - loss: 115.4949 - acc: 0.01 - ETA: 35s - loss: 114.0803 - acc: 0.01 - ETA: 34s - loss: 112.9304 - acc: 0.01 - ETA: 33s - loss: 111.9430 - acc: 0.01 - ETA: 32s - loss: 110.5766 - acc: 0.02 - ETA: 31s - loss: 110.4907 - acc: 0.01 - ETA: 30s - loss: 109.7073 - acc: 0.02 - ETA: 30s - loss: 108.8254 - acc: 0.02 - ETA: 29s - loss: 108.4154 - acc: 0.02 - ETA: 28s - loss: 107.5317 - acc: 0.02 -

 4608/11904 [==========>...................] - ETA: 26s - loss: 94.0842 - acc: 0.0000e+0 - ETA: 26s - loss: 94.2237 - acc: 0.0469    - ETA: 25s - loss: 95.3590 - acc: 0.033 - ETA: 25s - loss: 94.9788 - acc: 0.027 - ETA: 25s - loss: 95.0872 - acc: 0.075 - ETA: 25s - loss: 94.9268 - acc: 0.079 - ETA: 24s - loss: 94.0652 - acc: 0.068 - ETA: 24s - loss: 94.2937 - acc: 0.059 - ETA: 24s - loss: 94.1822 - acc: 0.053 - ETA: 24s - loss: 93.7876 - acc: 0.052 - ETA: 24s - loss: 93.9828 - acc: 0.048 - ETA: 24s - loss: 93.8766 - acc: 0.044 - ETA: 23s - loss: 94.1594 - acc: 0.040 - ETA: 23s - loss: 94.3275 - acc: 0.039 - ETA: 23s - loss: 94.2761 - acc: 0.044 - ETA: 22s - loss: 93.8577 - acc: 0.041 - ETA: 22s - loss: 94.0409 - acc: 0.039 - ETA: 22s - loss: 94.3448 - acc: 0.037 - ETA: 21s - loss: 94.5616 - acc: 0.035 - ETA: 21s - loss: 94.7306 - acc: 0.034 - ETA: 21s - loss: 94.8821 - acc: 0.032 - ETA: 21s - loss: 94.9883 - acc: 0.030 - ETA: 20s - loss: 95.0759 - acc: 0.029 - ETA: 20s - loss: 94.9697 

KeyboardInterrupt: 